In [5]:
import pandas as pd

In [4]:
import numpy as np

In [3]:
data = pd.read_csv("MICRODADOS.csv", delimiter=";",encoding="latin-1") ##,index_col=("ComorbidadeTabagismo","Municipio")
data

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido
0,2023-03-06,2023-03-06,2023-03-02,NaN,2023-03-06,NaN,NaN,2023-03-06,NaN,Descartados,...,Não,Não,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
1,2023-03-06,2023-03-03,2023-03-02,NaN,2023-03-06,NaN,NaN,2023-03-03,NaN,Descartados,...,Não,Não,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
2,2023-03-06,2023-03-06,2023-03-04,NaN,2023-03-06,NaN,NaN,2023-03-06,NaN,Descartados,...,Não,Não,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
3,2023-03-06,2023-03-06,2023-03-06,NaN,2023-03-06,NaN,NaN,2023-03-06,NaN,Descartados,...,Não,Não,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
4,2023-03-06,2023-03-06,2023-03-05,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Sim,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4724840,2020-02-27,2020-04-16,2020-02-15,2020-02-27,NaN,NaN,NaN,2020-04-16,NaN,Descartados,...,Não,Não,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado
4724841,2020-02-27,2020-04-16,2020-02-15,2020-02-27,NaN,NaN,NaN,2020-04-16,NaN,Descartados,...,Não,Não,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado
4724842,2020-02-27,2020-04-15,2020-02-25,2020-02-26,NaN,NaN,NaN,2020-04-15,NaN,Descartados,...,Não Informado,Sim,Não,Não Informado,Não Informado,Negativo,Não Informado,Não Informado,Não Informado,Não Informado
4724843,2020-02-25,2020-04-16,2020-02-13,2020-02-26,NaN,NaN,NaN,2020-04-16,2020-04-16,Descartados,...,Não,Sim,Não,Não,Não,Negativo,Não Informado,Não Informado,Não Informado,Não Informado
